In [2]:
import pandas as pd, numpy as np, json, os

In [3]:
base_url='http://www.cdep.ro'

In [4]:
os.chdir('E:/Onedrive - Lancaster University/datarepo/influence/ro')

In [5]:
nmembers=json.loads(open('data/nmembers.json','r').read())

In [6]:
img=json.loads(open('data/img.json','r').read())

In [7]:
os.chdir('E:/Skydrive/Github/influence/ro')

In [8]:
legs={'2016':['2016-12-21','2019-05-01'],
'2012':['2012-12-20','2016-12-21'],
'2008':['2008-12-19','2012-12-20'],
'2004':['2004-12-19','2008-12-13'],
'2000':['2000-12-15','2004-11-30'],
'1996':['1996-11-27','2000-11-30'],
'1992':['1992-10-28','1996-11-22'],
'1990':['1990-06-19','1992-10-16']}

In [9]:
import altair as alt

In [39]:
def extender(r,n=5):
    p={}
    for i in r:
        for j in range(n):
            p[i+j*' ']=r[i]+j*' '
    return p

In [66]:
pretty={'- din care declaraţii politice':'🧾 Nyilatkozatok',
        'Declaraţii politice depuse în scris':'🧾 Nyilatkozatok',
       'Luari de cuvânt':'🎙️ Felszólalások', 
        'Luari de cuvânt în BP':'🎙️ Felszólalások', 
        'Motiuni':'📚 Javaslatok', 
        'Proiecte de hotarâre initiate':'📚 Javaslatok', 
        'Propuneri legislative initiate':'📚 Javaslatok',
        'Sedințe':'💺 Ülések', 
        'Întrebari si interpelari':'🎙️ Felszólalások'}

In [80]:
efs=[]
for n in nmembers:
    m=nmembers[n]
    data=[]
    for pi in range(len(m['Link'])):
        p=m['Link'][pi]
        d={}
        d['Name']=m['Name']['full']
        d['Simple']=n
        d['Link']=p
        d['Session']=p[-10:-6]
        for a in m['Activity']:
            if type(m['Activity'][a])==list:
                g=m['Activity'][a]
            else:
                g=[m['Activity'][a]]
            for h in g:
                if h['start']==legs[d['Session']][0]: 
                    if pretty[a] not in d:d[pretty[a]]=0
                    d[pretty[a]]+=h['value']
        data.append(d)
    ef=pd.DataFrame(data)
    
    #append
    efs.append(ef)

In [144]:
efs2=pd.concat(efs).fillna(0).reset_index().drop('index',axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [178]:
j=efs2.T.astype(str).to_dict()
j2=list(j.values())

In [179]:
j3={}
for i in j2:
    if i['Name'] not in j3:j3[i['Name']]=[]
    j3[i['Name']].append(i)

In [180]:
open('json/member_activity.json','w').write(json.dumps(j3))

1479466

In [132]:
efs3=[]
cols=['🎙️ Felszólalások', '💺 Ülések',
       '📚 Javaslatok', '🧾 Nyilatkozatok']
for c in cols:
    ef=efs2.set_index(['Name', 'Session', 'Simple'])[[c]]
    ef.columns=['Value']
    ef['Type']=c
    efs3.append(ef)
efs3=pd.concat(efs3).reset_index()

In [138]:
j=efs3.T.astype(str).to_dict()
open('json/member_activity2.json','w').write(json.dumps(list(j.values())))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  """Entry point for launching an IPython kernel.


293716